In [2]:
list_patches = [
    "7.20", "7.20b", "7.20c", "7.20d", "7.20e", 
    "7.21", "7.21b", "7.21c", "7.21d", 
    "7.22", "7.22b", "7.22c", "7.22d", "7.22e", "7.22f", "7.22g", "7.22h",
    "7.23", "7.23a", "7.23b", "7.23c", "7.23d", "7.23e", "7.23f", 
    "7.24", "7.24b", 
    "7.25", "7.25a", "7.25b", "7.25c", 
    "7.26", "7.26a", "7.26b", "7.26c", 
    "7.27", "7.27a", "7.27b", "7.27c", "7.27d", 
    "7.28", "7.28a", "7.28b", "7.28c", 
    "7.29", "7.29b", "7.29c", "7.29d", 
    "7.30", "7.30b", "7.30c", "7.30d", "7.30e", 
    "7.31", "7.31b", "7.31c", "7.31d",
    "7.32", "7.32b", "7.32c", "7.32d", "7.32e",
    "7.33", "7.33b", "7.33c", "7.33d", "7.33e",
    "7.34", "7.34b", "7.34c", "7.34d", "7.34e",
    "7.35", "7.35b", "7.35c", "7.35d", 
    "7.36", "7.36a", "7.36b", "7.36c", 
    "7.37", "7.37b", "7.37c", "7.37d"
]

In [74]:
# import requests

# for p in list_patches:
# # URL of the Dota2 API endpoint
#     url = f'https://www.dota2.com/datafeed/patchnotes?version={p}&language=english'
#     response = requests.get(url)
#     data = response.json()
#     # Save into a file
#     # Save the JSON response to a file for JSONLoader to read
#     with open(f"patchnotes/{p}.json", "w") as f:
#         import json
#         json.dump(data, f)

In [1]:
import requests
import json

with open("patchnotes/7.35.json", 'r') as file:
  json_data = json.load(file)

def add_default_hero_values(data):
  for hero in data.get("heroes", []):
    # Add default talent_notes if not present
    if "talent_notes" not in hero:
      hero["talent_notes"] = ["No updates/changes"]
    if "hero_notes" not in hero:
      hero["hero_notes"] = ["No updates/changes"]
    if "abilities" not in hero:
      hero["abilities"] = [{'ability_id': 5007}, {"ability_notes":["No updates/changes"]}]

add_default_hero_values(json_data)

# Save to a temporary file
with open('patchnotes/temp_7.35.json', 'w') as f:
  json.dump(json_data, f)

In [2]:
from langchain_community.document_loaders import JSONLoader
import ast

In [3]:
# Heroes Abilities
def extract_metadata(record: dict, metadata: dict) -> dict:
  metadata["hero_id"] = record.get("hero_id")
  abilities = record.get("abilities", [])
  if abilities:
    metadata["ability_id"] = abilities[0].get("ability_id")
  else:
    metadata["ability_id"] = None
  metadata["category"] = "heroes-abilities"
  return metadata

loader_heroes_abilities = JSONLoader(
  file_path="patchnotes/temp_7.35.json",
  jq_schema=".heroes[]",
  content_key="abilities",
  text_content=False,
  metadata_func=extract_metadata
)
docs_heroes_abilities = loader_heroes_abilities.load()
docs_heroes_abilities


[Document(metadata={'source': 'D:\\files_main\\projects_ml\\rag-dota2-patch-updates-librarian\\patchnotes\\temp_7.35.json', 'seq_num': 1, 'hero_id': 1, 'ability_id': 5007, 'category': 'heroes-abilities'}, page_content="[{'ability_id': 5007}, {'ability_notes': ['No updates/changes']}]"),
 Document(metadata={'source': 'D:\\files_main\\projects_ml\\rag-dota2-patch-updates-librarian\\patchnotes\\temp_7.35.json', 'seq_num': 2, 'hero_id': 2, 'ability_id': 5007, 'category': 'heroes-abilities'}, page_content="[{'ability_id': 5007, 'ability_notes': [{'indent_level': 1, 'note': 'Bonus Armor decreased from 25 to 16/19/22/25'}]}, {'ability_id': 5010, 'ability_notes': [{'indent_level': 1, 'note': 'Temporary boost on kill no longer provides +20/30/40 Attack Speed'}, {'indent_level': 1, 'note': 'Temporary boost on kill now provides +20/25/30 Armor'}]}]"),
 Document(metadata={'source': 'D:\\files_main\\projects_ml\\rag-dota2-patch-updates-librarian\\patchnotes\\temp_7.35.json', 'seq_num': 3, 'hero_id'

In [4]:
# Heroes Talents
def extract_metadata(record: dict, metadata: dict) -> dict:
  metadata["hero_id"] = record.get("hero_id")
  metadata["category"] = "heroes-talents"
  return metadata

loader_heroes_talents = JSONLoader(
  file_path="patchnotes/temp_7.35.json",
  jq_schema=".heroes[]",
  content_key="talent_notes",
  text_content=False,
  metadata_func=extract_metadata
)
docs_heroes_talents = loader_heroes_talents.load()
# Check results
docs_heroes_talents

[Document(metadata={'source': 'D:\\files_main\\projects_ml\\rag-dota2-patch-updates-librarian\\patchnotes\\temp_7.35.json', 'seq_num': 1, 'hero_id': 1, 'category': 'heroes-talents'}, page_content="[{'indent_level': 1, 'note': 'Level 15 Talent Max Mana Mana Burn increased from +0.6% to +1%'}, {'indent_level': 1, 'note': 'Level 20 Talent Mana Void Damage Multiplier increased from +0.15 to +0.2'}]"),
 Document(metadata={'source': 'D:\\files_main\\projects_ml\\rag-dota2-patch-updates-librarian\\patchnotes\\temp_7.35.json', 'seq_num': 2, 'hero_id': 2, 'category': 'heroes-talents'}, page_content='[{\'indent_level\': 1, \'note\': "Level 10 Talent +10 Berserker\'s Call Armor replaced with +4s Culling Blade Speed Bonus Duration"}]'),
 Document(metadata={'source': 'D:\\files_main\\projects_ml\\rag-dota2-patch-updates-librarian\\patchnotes\\temp_7.35.json', 'seq_num': 3, 'hero_id': 3, 'category': 'heroes-talents'}, page_content="['No updates/changes']"),
 Document(metadata={'source': 'D:\\files_m

In [5]:
# Heroes Base
def extract_metadata(record: dict, metadata: dict) -> dict:
  metadata["hero_id"] = record.get("hero_id")
  metadata["category"] = "heroes-base"
  return metadata

loader_heroes_base = JSONLoader(
  file_path="patchnotes/temp_7.35.json",
  jq_schema=".heroes[]",
  content_key="hero_notes",
  text_content=False,
  metadata_func=extract_metadata
)
docs_heroes_base = loader_heroes_base.load()
# Check results
docs_heroes_base

[Document(metadata={'source': 'D:\\files_main\\projects_ml\\rag-dota2-patch-updates-librarian\\patchnotes\\temp_7.35.json', 'seq_num': 1, 'hero_id': 1, 'category': 'heroes-base'}, page_content="['No updates/changes']"),
 Document(metadata={'source': 'D:\\files_main\\projects_ml\\rag-dota2-patch-updates-librarian\\patchnotes\\temp_7.35.json', 'seq_num': 2, 'hero_id': 2, 'category': 'heroes-base'}, page_content="[{'indent_level': 1, 'note': 'Base Armor increased by 1'}]"),
 Document(metadata={'source': 'D:\\files_main\\projects_ml\\rag-dota2-patch-updates-librarian\\patchnotes\\temp_7.35.json', 'seq_num': 3, 'hero_id': 3, 'category': 'heroes-base'}, page_content="['No updates/changes']"),
 Document(metadata={'source': 'D:\\files_main\\projects_ml\\rag-dota2-patch-updates-librarian\\patchnotes\\temp_7.35.json', 'seq_num': 4, 'hero_id': 4, 'category': 'heroes-base'}, page_content="['No updates/changes']"),
 Document(metadata={'source': 'D:\\files_main\\projects_ml\\rag-dota2-patch-updates-

In [13]:
# Items
def extract_metadata(record: dict, metadata: dict) -> dict:
  metadata["item_id"] = record.get("ability_id")
  metadata["category"] = "items"
  metadata["patch"] = "7.35"
  return metadata

loader_items = JSONLoader(
  file_path="patchnotes/temp_7.35.json",
  jq_schema=".items[]",
  content_key="ability_notes",
  is_content_key_jq_parsable=False,
  text_content=False,
  metadata_func=extract_metadata
)
docs_items = loader_items.load()

# Items Mapper
items_url = 'https://api.opendota.com/api/constants/item_ids'
items_response = requests.get(items_url)
dict_item_map = items_response.json()
dict_item_map = {key: value.replace('_', ' ') for key, value in dict_item_map.items()}

def replace_item_id_with_name(documents, item_id_map):
  # Loop through each document and replace the item_id with its corresponding name
  for doc in documents:
    item_id = doc.metadata.get('item_id')
    # Convert the item_id to a string to match the dictionary keys
    item_id_str = str(item_id)
    # Replace item_id with corresponding value from the dictionary, if available
    if item_id_str in item_id_map:
      doc.metadata['item_id'] = item_id_map[item_id_str]  # Replace with the item name4
    doc.page_content = f"Item: {doc.metadata['item_id']}. {doc.page_content}"  
  return documents

docs_items = replace_item_id_with_name(docs_items, dict_item_map)
docs_items

[Document(metadata={'source': 'D:\\files_main\\projects_ml\\rag-dota2-patch-updates-librarian\\patchnotes\\temp_7.35.json', 'seq_num': 1, 'item_id': 'ring of tarrasque', 'category': 'items', 'patch': '7.35'}, page_content="Item: ring of tarrasque. [{'indent_level': 2, 'note': 'Now is a basic Secret Shop item. Cost: 1800'}, {'indent_level': 2, 'note': 'Provides +12 Health Regen'}]"),
 Document(metadata={'source': 'D:\\files_main\\projects_ml\\rag-dota2-patch-updates-librarian\\patchnotes\\temp_7.35.json', 'seq_num': 2, 'item_id': 'tiara of selemene', 'category': 'items', 'patch': '7.35'}, page_content="Item: tiara of selemene. [{'indent_level': 2, 'note': 'New basic Secret Shop item. Cost: 1800'}, {'indent_level': 2, 'note': 'Provides +6 Mana Regen'}]"),
 Document(metadata={'source': 'D:\\files_main\\projects_ml\\rag-dota2-patch-updates-librarian\\patchnotes\\temp_7.35.json', 'seq_num': 3, 'item_id': 'angels demise', 'category': 'items', 'patch': '7.35'}, page_content='Item: angels demi

In [35]:
docs_all = docs_heroes_base + docs_heroes_abilities + docs_heroes_talents + docs_items

In [7]:
def ensure_page_content_is_string(documents):
  # Loop through each document and make sure page_content is a string
  for doc in documents:
      if not isinstance(doc.page_content, str):
          doc.page_content = str(doc.page_content)  # Convert to string if it's not already a string
  return documents

# Ensure all page_content fields are strings
docs_all = ensure_page_content_is_string(docs_all)

NameError: name 'docs_all' is not defined

In [56]:
docs_all

[Document(metadata={'source': 'D:\\files_main\\projects_ml\\rag-dota2-patch-updates-librarian\\patchnotes\\temp_7.35.json', 'seq_num': 1, 'hero_id': 1, 'category': 'heroes-base'}, page_content="['No updates/changes']"),
 Document(metadata={'source': 'D:\\files_main\\projects_ml\\rag-dota2-patch-updates-librarian\\patchnotes\\temp_7.35.json', 'seq_num': 2, 'hero_id': 2, 'category': 'heroes-base'}, page_content="[{'indent_level': 1, 'note': 'Base Armor increased by 1'}]"),
 Document(metadata={'source': 'D:\\files_main\\projects_ml\\rag-dota2-patch-updates-librarian\\patchnotes\\temp_7.35.json', 'seq_num': 3, 'hero_id': 3, 'category': 'heroes-base'}, page_content="['No updates/changes']"),
 Document(metadata={'source': 'D:\\files_main\\projects_ml\\rag-dota2-patch-updates-librarian\\patchnotes\\temp_7.35.json', 'seq_num': 4, 'hero_id': 4, 'category': 'heroes-base'}, page_content="['No updates/changes']"),
 Document(metadata={'source': 'D:\\files_main\\projects_ml\\rag-dota2-patch-updates-

In [18]:
from typing import Sequence

import bs4
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.messages import AIMessage, BaseMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, StateGraph
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [19]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs_items)
vectorstore = InMemoryVectorStore.from_documents(
    documents=splits, embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()

In [20]:
### Contextualize question ###
contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

### Answer question ###
system_prompt = (
    "You are an assistant for question-answering tasks about DOTA2 patch notes. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)


### Statefully manage chat history ###
class State(TypedDict):
    input: str
    chat_history: Annotated[Sequence[BaseMessage], add_messages]
    context: str
    answer: str


def call_model(state: State):
    response = rag_chain.invoke(state)
    return {
        "chat_history": [
            HumanMessage(state["input"]),
            AIMessage(response["answer"]),
        ],
        "context": response["context"],
        "answer": response["answer"],
    }


workflow = StateGraph(state_schema=State)
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [21]:
docs_items

[Document(metadata={'source': 'D:\\files_main\\projects_ml\\rag-dota2-patch-updates-librarian\\patchnotes\\temp_7.35.json', 'seq_num': 1, 'item_id': 'ring of tarrasque', 'category': 'items', 'patch': '7.35'}, page_content="Item: ring of tarrasque. [{'indent_level': 2, 'note': 'Now is a basic Secret Shop item. Cost: 1800'}, {'indent_level': 2, 'note': 'Provides +12 Health Regen'}]"),
 Document(metadata={'source': 'D:\\files_main\\projects_ml\\rag-dota2-patch-updates-librarian\\patchnotes\\temp_7.35.json', 'seq_num': 2, 'item_id': 'tiara of selemene', 'category': 'items', 'patch': '7.35'}, page_content="Item: tiara of selemene. [{'indent_level': 2, 'note': 'New basic Secret Shop item. Cost: 1800'}, {'indent_level': 2, 'note': 'Provides +6 Mana Regen'}]"),
 Document(metadata={'source': 'D:\\files_main\\projects_ml\\rag-dota2-patch-updates-librarian\\patchnotes\\temp_7.35.json', 'seq_num': 3, 'item_id': 'angels demise', 'category': 'items', 'patch': '7.35'}, page_content='Item: angels demi

In [22]:
from langchain_core.messages import AIMessage, HumanMessage

chat_history = []

question = "What is the tiara of selemene in 7.34 patch?"
ai_msg_1 = rag_chain.invoke({"input": question, "chat_history": chat_history})
chat_history.extend(
    [
      HumanMessage(content=question),
      AIMessage(content=ai_msg_1["answer"]),
    ]
)

# second_question = "What are common ways of doing it?"
# ai_msg_2 = rag_chain.invoke({"input": second_question, "chat_history": chat_history})

print(ai_msg_1["answer"])

In the 7.34 patch, the Tiara of Selemene is a new basic Secret Shop item that costs 1800 gold and provides +6 Mana Regen.


In [23]:

### Construct retriever ###
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = InMemoryVectorStore.from_documents(
    documents=splits, embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()


### Contextualize question ###
contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)


### Answer question ###
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)


### Statefully manage chat history ###
class State(TypedDict):
    input: str
    chat_history: Annotated[Sequence[BaseMessage], add_messages]
    context: str
    answer: str


def call_model(state: State):
    response = rag_chain.invoke(state)
    return {
        "chat_history": [
            HumanMessage(state["input"]),
            AIMessage(response["answer"]),
        ],
        "context": response["context"],
        "answer": response["answer"],
    }


workflow = StateGraph(state_schema=State)
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [6]:
from langchain_core.messages import AIMessage, HumanMessage

chat_history = []

question = "What is Task Decomposition?"
ai_msg_1 = rag_chain.invoke({"input": question, "chat_history": chat_history})
chat_history.extend(
    [
        HumanMessage(content=question),
        AIMessage(content=ai_msg_1["answer"]),
    ]
)

second_question = "What are common ways of doing it?"
ai_msg_2 = rag_chain.invoke({"input": second_question, "chat_history": chat_history})

print(ai_msg_2["answer"])

Common ways of task decomposition include using simple prompting techniques, such as asking for "Steps for XYZ" or "What are the subgoals for achieving XYZ?" Additionally, task-specific instructions can be employed, like "Write a story outline" for writing tasks, or human inputs can guide the decomposition process.


In [ ]:
#Sample